# Basic CNN Experiment

Simple CNN to establish baseline performance on FER2013 dataset.

In [ ]:
!pip install -q torch torchvision wandb pandas numpy matplotlib tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
from tqdm import tqdm

## Download Data from Kaggle

In [ ]:
# Mount Drive for kaggle.json
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setup Kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

In [ ]:
# Check files
!ls -la *.csv *.gz

## Dataset Class

In [ ]:
class FERDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get pixels
        pixels = self.data.iloc[idx, 1]  # pixels column
        pixels = np.array(pixels.split(' '), dtype='float32').reshape(48, 48)
        pixels = pixels / 255.0  # normalize
        
        # Convert to 3 channel
        image = np.stack([pixels] * 3, axis=2)
        
        # Get label
        label = int(self.data.iloc[idx, 0])  # emotion column
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

## Simple CNN Model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 7)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        x = x.view(-1, 128 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## Setup Data

In [ ]:
# Check which file we have
if os.path.exists('train.csv'):
    print("Using train.csv")
    train_file = 'train.csv'
elif os.path.exists('fer2013.csv'):
    print("Using fer2013.csv")
    train_file = 'fer2013.csv'
else:
    print("No data file found!")

In [ ]:
# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create dataset
dataset = FERDataset(train_file, transform=transform)

# Split into train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")

## Train

In [ ]:
# Initialize wandb
wandb.login()
wandb.init(project="fer-challenge", name="basic-cnn")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
epochs = 20

for epoch in range(epochs):
    # Train
    model.train()
    train_loss = 0
    train_correct = 0
    
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(labels).sum().item()
    
    # Validate
    model.eval()
    val_loss = 0
    val_correct = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
    
    # Calculate metrics
    train_acc = 100. * train_correct / len(train_dataset)
    val_acc = 100. * val_correct / len(val_dataset)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
    
    # Log to wandb
    wandb.log({
        "train_acc": train_acc,
        "val_acc": val_acc,
        "train_loss": train_loss/len(train_loader),
        "val_loss": val_loss/len(val_loader)
    })

In [ ]:
wandb.finish()
print("Training complete!")